In [ ]:
import json
import csv
import numpy as np
import pandas as pd
from urllib.parse import quote  
import scipy.stats as stats
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option("display.max_rows",None)

In [ ]:
fileList={
    "DF1":"botometer-feedback-2019",
    "DF2":"botwiki-2019",
    "DF3":"celebrity-2019",
    "DF4":"cresci-rtbust-2019",
    "DF5":"cresci-stock-2018",
    "DF6":"gilani-2017",
    "DF7":"midterm-2018",
    "DF8":"political-bots-2019",
    "DF9":"pronbots-2019",
    "DF10":"vendor-purchased-2019",
    "DF11":"verified-2019",
    "DF12":"pron-celebrity",#DF3+DF9
    "DF13":"political-feedback",#DF1+DF8
    "DF14":"botwiki-verified",#DF2+DF11
    "DF15":"vendor-verified",#DF10+DF11
}


remainForTesting={
    "DF4":"cresci-rtbust-2019",
    "DF5":"cresci-stock-2018",
    "DF6":"gilani-2017",
    "DF7":"midterm-2018",
    "DF12":"pron-celebrity",#DF3+DF9
    "DF13":"political-feedback",#DF1+DF8
    "DF14":"botwiki-verified",#DF2+DF11
    "DF15":"vendor-verified",#DF10+DF11
}
["caveree","varol-icwsm","cresci-2017"]

['caveree', 'varol-icwsm', 'cresci-2017']

In [ ]:
from difflib import SequenceMatcher
from nltk import everygrams

def likehood(s):
  Bi_list=["".join(item) for item in list(everygrams(s, 2, 2))]
  K=sum(list({i:Bi_list.count(i) for i in Bi_list}.values()))
  CBi= len(np.unique(Bi_list, return_counts=True)[1])
  return CBi/K if K>0 else 1
  
def entropy(string):
    prob = [ float(string.count(c)) / len(string) for c in dict.fromkeys(list(string)) ]
    entropy = - sum([ p * math.log(p) / math.log(2.0) for p in prob ])
    return entropy/len(string) if len(string)>0 else 1
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


In [ ]:
def drop_data(data):
  data.drop(columns=["lang","time_zone","follow_request_sent","utc_offset","notifications","following"], axis=1, inplace=True);
  data.drop(columns=["protected","contributors_enabled"], axis=1, inplace=True);
  data.drop(columns=["is_translator","is_translation_enabled"], axis=1, inplace=True);
  data.drop(columns=["profile_sidebar_fill_color","profile_link_color","profile_sidebar_border_color","profile_background_color","profile_text_color"], axis=1, inplace=True);
  data.drop(columns=["id","id_str","twitter_created_at","created_at"], axis=1, inplace=True);
  data.drop(columns=["translator_type"], axis=1, inplace=True);
  data.drop(columns=["profile_background_image_url"], axis=1, inplace=True);
  data.drop(columns=["profile_image_url_https"], axis=1, inplace=True);
  data.drop(columns=["profile_image_url"], axis=1, inplace=True);

In [ ]:
def preprocess (data):

  data['created_at'] = pd.to_datetime(data['created_at'])
  data['twitter_created_at'] = pd.to_datetime(data['twitter_created_at'])
  data['diff_days'] = (data['twitter_created_at']-data['created_at']).dt.days
  data['diff_days'].replace(0, 1,inplace=True)

  data["name"].fillna("",inplace=True)
  data["description"].fillna("",inplace=True)
  data['name_length']=data['name'].map(lambda x: len(x))
  data['screen_name_length']=data['screen_name'].map(lambda x: len(str(x)))
  data['description_length']=data['description'].map(lambda x: len(x))

  data['num_digits_in_name']=data['name'].map(lambda x: sum(c.isdigit() for c in x))
  data['num_digits_in_screen_name']=data['screen_name'].map(lambda x: sum(c.isdigit() for c in str(x)))

  data['tweet_freq']=data['statuses_count']/data['diff_days']
  data['followers_growth_rate']=data['followers_count']/data['diff_days']
  data['friends_growth_rate']=data['friends_count']/data['diff_days']
  data['favourites_growth_rate']=data['favourites_count']/data['diff_days']
  data['listed_growth_rate']=data['listed_count']/data['diff_days']
  data['followers_friends_ratio']=data['followers_count']/data['diff_days']
  data['Name_entropy']=data.apply(lambda row: entropy(str(row['name'])),axis=1)
  data['Screen_name_entropy']=data.apply(lambda row: entropy(str(row['screen_name'])),axis=1)
  data['Screen_name_freq']=data.apply(lambda row: likehood(str(row['screen_name'])),axis=1)
  data['Name_similarity']=data.apply(lambda row: similar(str(row['screen_name']),str(row['name'])),axis=1)
  data['Name_freq']=data.apply(lambda row: likehood(str(row['name'])) if len(row['name'])>0 else 1,axis=1)

In [ ]:
def convert(data):
  data["has_extended_profile"]=np.where(data["has_extended_profile"].isnull(),False,data["has_extended_profile"])
  list=["profile_background_tile","default_profile","profile_use_background_image","default_profile_image","geo_enabled","verified","has_extended_profile"]
  data[list]= data[list].applymap({True: 1, False: 0}.get)
  data["profile_background_image_url_https"]=np.where(data["profile_background_image_url_https"].isnull(),0,1)
  data["_url"]=np.where(data["url"].isnull(),0,1)
  data["_location"]=np.where(data["location"].isnull(),0,1)
  data["profile_banner_url"]=np.where(data["profile_banner_url"].isnull(),0,1)
  data["entities"]=np.where(data["entities.url.urls"]=="{'description': {'urls': []}}",0,1)
  data.drop(columns=["url","location","name","description","screen_name","entities.description.urls","entities.url.urls"], axis=1, inplace=True);

In [ ]:
# It work for all datasets beside DS7
for index in range(1,12):
  if index==7:
    continue
  data = pd.read_csv(f'PATH/Paper-Dataset-Preparing/DF{index}.csv.gz',compression='gzip',lineterminator='\n');
  preprocess(data)
  drop_data(data)
  convert(data)
  print(len(data.columns))
  data.to_csv(f'PATH/Paper-Dataset-Preparing/DS{index}.csv.gz', compression='gzip',index=False)

In [ ]:
def preprocessDF7 (data):

  data['user_created_at'] = pd.to_datetime(data['user_created_at'])
  data['probe_timestamp'] = pd.to_datetime(data['probe_timestamp'])
  data['diff_days'] = (data['probe_timestamp']-data['user_created_at']).dt.days
  data['diff_days'].replace(0, 1,inplace=True)

  data["name"].fillna("",inplace=True)
  data["description"].fillna("",inplace=True)
  data['name_length']=data['name'].map(lambda x: len(x))
  data['screen_name_length']=data['screen_name'].map(lambda x: len(str(x)))
  data['description_length']=data['description'].map(lambda x: len(x))

  data['num_digits_in_name']=data['name'].map(lambda x: sum(c.isdigit() for c in x))
  data['num_digits_in_screen_name']=data['screen_name'].map(lambda x: sum(c.isdigit() for c in str(x)))

  data['tweet_freq']=data['statuses_count']/data['diff_days']
  data['followers_growth_rate']=data['followers_count']/data['diff_days']
  data['friends_growth_rate']=data['friends_count']/data['diff_days']
  data['favourites_growth_rate']=data['favourites_count']/data['diff_days']
  data['listed_growth_rate']=data['listed_count']/data['diff_days']
  data['followers_friends_ratio']=data['followers_count']/data['diff_days']
  data['Name_entropy']=data.apply(lambda row: entropy(str(row['name'])),axis=1)
  data['Screen_name_entropy']=data.apply(lambda row: entropy(str(row['screen_name'])),axis=1)
  data['Screen_name_freq']=data.apply(lambda row: likehood(str(row['screen_name'])),axis=1)
  data['Name_similarity']=data.apply(lambda row: similar(str(row['screen_name']),str(row['name'])),axis=1)
  data['Name_freq']=data.apply(lambda row: likehood(str(row['name'])) if len(row['name'])>0 else 1,axis=1)

In [ ]:
def drop_dataDF7(data):
  data.drop(columns=["protected","id","lang","probe_timestamp","user_created_at","tid"], axis=1, inplace=True);

In [ ]:
def convertDF7(data):
  data["has_extended_profile"]=False
  data["profile_background_tile"]=False
  data["default_profile_image"]=False
  
  list=["default_profile","profile_use_background_image","geo_enabled","verified","has_extended_profile"]
  data[list]= data[list].applymap({True: 1, False: 0}.get)
  data["profile_background_image_url_https"]=0
  data["_url"]=np.where(data["url"].isnull(),0,1)
  data["_location"]=0
  data["profile_banner_url"]=0
  data["entities"]=0
  data.drop(columns=["url","name","description","screen_name"], axis=1, inplace=True);

In [ ]:
# ds1 = pd.read_csv(f'PATH/Dataset_Classified_With_BotometerLite.csv.gz',compression='gzip',lineterminator='\n');
ds2 = pd.read_csv(f'PATH/step1.csv.gz',compression='gzip',lineterminator='\n');

print(len(ds2))
print((ds2.columns))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (18,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


1246936
Index(['url', 'listed_count', 'lang', 'statuses_count', 'favourites_count',
       'profile_background_image_url', 'profile_sidebar_fill_color', 'id',
       'created_at', 'protected', 'profile_background_tile', 'location',
       'profile_background_image_url_https', 'profile_link_color',
       'profile_sidebar_border_color', 'friends_count', 'id_str', 'time_zone',
       'has_extended_profile', 'followers_count', 'name',
       'is_translation_enabled', 'profile_image_url', 'follow_request_sent',
       'utc_offset', 'profile_image_url_https', 'notifications', 'description',
       'withheld_in_countries', 'profile_background_color', 'is_translator',
       'default_profile', 'screen_name', 'profile_use_background_image',
       'default_profile_image', 'profile_banner_url', 'profile_text_color',
       'geo_enabled', 'verified', 'translator_type', 'entities',
       'contributors_enabled', 'following', 'user_created_at', 'class',
       'diff_days', 'name_length', 'screen_n

' for index in range(1,17):\n    data = pd.read_csv(f\'/content/drive/MyDrive/ML/DATA/MetaData/Test/DS{index}.csv.gz\',compression=\'gzip\',lineterminator=\'\n\');\n    print(len(data[data["class"]=="human"]))\n    print(len(data[data["class"]=="bot"]))\n    print("------------------------") '

In [ ]:
tdata = pd.read_csv(f'PATH/t-step1.csv.gz',compression='gzip',lineterminator='\n');
# print(len(tdata))
# tdata.nunique()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (18,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data = pd.read_csv(f'PATH/step1.csv.gz',compression='gzip',lineterminator='\n');
# print(len(data))
# data.nunique()

In [ ]:
result = pd.concat([data, tdata])
# print(len(result))
# result.nunique().sort_index(ascending=True)

In [ ]:
result["withheld_in_countries"].value_counts(dropna=False)

NaN             2692072
['TR']               69
['RU']               14
['DE']               12
['FR']                8
['IN']                7
['DE', 'FR']          2
['DE', 'RU']          1
Name: withheld_in_countries, dtype: int64

In [ ]:
result.nunique(dropna=False).sort_index(ascending=True)

Name_entropy                            19943
Name_freq                                 597
Name_similarity                           471
Screen_name_entropy                       669
Screen_name_freq                           54
class                                     740
classify                              1445250
contributors_enabled                        1
created_at                            2680434
default_profile                             2
default_profile_image                       2
description                           2113005
description_length                        227
diff_days                                5204
entities                               692623
favourites_count                       168635
favourites_growth_rate                2274298
follow_request_sent                         1
followers_count                         47876
followers_friends_ratio               1379904
followers_growth_rate                 1379904
following                         

In [ ]:
# preprocessDF7(data)
# drop_dataDF7(data)
# convertDF7(data)
# print(len(data.columns))
# data.head()
# data.to_csv(f'PATH/DS7.csv.gz', compression='gzip',index=False)

In [ ]:
# DS3 has one more column than other Datasets
data = pd.read_csv(f'PATH/DS3.csv.gz',compression='gzip',lineterminator='\n');
# data.drop(columns=["withheld_in_countries"], axis=1, inplace=True);
# data.to_csv(f'PATH/DS3.csv.gz', compression='gzip',index=False)
data.head()

,has_extended_profile,profile_use_background_image,default_profile_image,profile_background_image_url_https,verified,followers_count,listed_count,statuses_count,friends_count,geo_enabled,profile_banner_url,profile_background_tile,favourites_count,default_profile,class,diff_days,name_length,screen_name_length,description_length,num_digits_in_name,num_digits_in_screen_name,tweet_freq,followers_growth_rate,friends_growth_rate,favourites_growth_rate,listed_growth_rate,followers_friends_ratio,Name_entropy,Screen_name_entropy,Screen_name_freq,Name_similarity,Name_freq,_url,_location,entities
0,0,1,0,1,0,3042228,3687,69200,29182,0,1,0,16,0,NaN,2346,11,11,124,0,0,29.497016,1296.772379,12.439045,0.006820,1.571611,1296.772379,0.297965,0.297965,1.0,0.181818,1.000000,1,1,1
1,1,0,0,1,1,651238,1761,39411,665,0,0,1,13561,0,human,3519,11,11,117,0,0,11.199488,185.063370,0.188974,3.853652,0.500426,185.063370,0.264907,0.264907,1.0,1.000000,1.000000,1,1,1
2,0,0,0,1,1,265102,783,41187,943,0,1,0,11365,0,human,3204,19,13,40,0,0,12.854869,82.740949,0.294320,3.547129,0.244382,82.740949,0.172980,0.232845,1.0,0.562500,0.944444,1,1,1
3,0,1,0,1,1,2724932,3164,310,14,0,1,0,76,1,human,1501,12,14,86,0,0,0.206529,1815.411059,0.009327,0.050633,2.107928,1815.411059,0.284858,0.217082,1.0,0.384615,1.000000,1,0,1
4,0,1,0,1,0,24329,67,11989,1379,1,1,1,2696,0,NaN,2110,13,8,54,0,0,5.681991,11.530332,0.653555,1.277725,0.031754,11.530332,0.272815,0.375000,1.0,0.476190,1.000000,1,1,1


In [ ]:
# There are some datasets that the [class] column contain [null]. We remove those data 
# for index in range(1,12):
#   data = pd.read_csv(f'PATH/DS{index}.csv.gz',compression='gzip',lineterminator='\n');
#   if len(data[data['class'].isnull()])>0:
#     print(data["class"].value_counts(dropna=False))
#     print("___________________________\n")
# data = data[data['class'].notna()]
# data.to_csv(f'PATH/DS{index}.csv.gz', compression='gzip',index=False)

In [ ]:
# Combining DS3 and DS9

# dataset1 = pd.read_csv(f'PATH/DS3.csv.gz',compression='gzip',lineterminator='\n');
# dataset2 = pd.read_csv(f'PATH/DS9.csv.gz',compression='gzip',lineterminator='\n');

# data=pd.concat([dataset1, dataset2], ignore_index=True, sort=True)
# data = data.sample(frac=1).reset_index(drop=True)
# data.to_csv(f'PATH/DS12.csv.gz', compression='gzip',index=False)

,Name_entropy,Name_freq,Name_similarity,Screen_name_entropy,Screen_name_freq,_location,_url,class,default_profile,default_profile_image,description_length,diff_days,entities,favourites_count,favourites_growth_rate,followers_count,followers_friends_ratio,followers_growth_rate,friends_count,friends_growth_rate,geo_enabled,has_extended_profile,listed_count,listed_growth_rate,name_length,num_digits_in_name,num_digits_in_screen_name,profile_background_image_url_https,profile_background_tile,profile_banner_url,profile_use_background_image,screen_name_length,statuses_count,tweet_freq,verified
0,0.227286,0.923077,0.296296,0.260981,1.0,1,1,bot,0,0,57,2563,1,125,0.048771,31,0.012095,0.012095,0,0.0,0,0,0,0.0,14,0,0,1,0,1,1,13,48,0.018728,0
1,0.234375,1.000000,0.230769,0.312193,1.0,0,0,bot,1,0,55,680,1,89,0.130882,28,0.041176,0.041176,0,0.0,0,0,0,0.0,16,0,3,0,0,1,1,10,27,0.039706,0
2,0.297965,1.000000,0.111111,0.360234,1.0,0,0,bot,1,0,59,731,1,282,0.385773,12,0.016416,0.016416,0,0.0,0,0,0,0.0,11,0,0,0,0,1,1,7,111,0.151847,0
3,0.226562,1.000000,0.142857,0.270969,1.0,0,0,bot,1,0,65,1614,1,741,0.459108,83,0.051425,0.051425,0,0.0,0,0,0,0.0,16,0,1,1,0,1,1,12,168,0.104089,0
4,0.207126,0.928571,0.357143,0.260981,1.0,0,0,bot,1,0,47,1438,1,139,0.096662,6,0.004172,0.004172,0,0.0,0,0,0,0.0,15,0,2,1,0,1,1,13,26,0.018081,0


In [ ]:
# dataset1 = pd.read_csv(f'PATH/DS1.csv.gz',compression='gzip',lineterminator='\n');
# dataset2 = pd.read_csv(f'PATH/DS8.csv.gz',compression='gzip',lineterminator='\n');

# data=pd.concat([dataset1, dataset2], ignore_index=True, sort=True)
# data = data.sample(frac=1).reset_index(drop=True)

# data.to_csv(f'PATH/DS13.csv.gz', compression='gzip',index=False)

In [ ]:
# dataset1 = pd.read_csv(f'PATH/DS1.csv.gz',compression='gzip',lineterminator='\n');
# dataset2 = pd.read_csv(f'PATH/DS10.csv.gz',compression='gzip',lineterminator='\n');
# dataset3 = pd.read_csv(f'PATH/DS11.csv.gz',compression='gzip',lineterminator='\n');

# data=pd.concat([dataset1, np.array_split(dataset3, 2)[0]], ignore_index=True, sort=True)
# data = data.sample(frac=1).reset_index(drop=True)
# data.to_csv(f'PATH/DS14.csv.gz', compression='gzip',index=False)

# data=pd.concat([dataset2, np.array_split(dataset3, 2)[1]], ignore_index=True, sort=True)
# data = data.sample(frac=1).reset_index(drop=True)
# data.to_csv(f'PATH/DS15.csv.gz', compression='gzip',index=False)

In [ ]:
# Make the Test dataset columns arragment as Main dataset
columns=['listed_count', 'statuses_count', 'favourites_count',
       'profile_background_tile', 'profile_background_image_url_https',
       'friends_count', 'has_extended_profile', 'followers_count',
       'default_profile', 'profile_use_background_image',
       'default_profile_image', 'profile_banner_url', 'geo_enabled',
       'verified', 'entities', 'class', 'diff_days', 'name_length',
       'screen_name_length', 'description_length', 'num_digits_in_name',
       'num_digits_in_screen_name', 'tweet_freq', 'followers_growth_rate',
       'friends_growth_rate', 'favourites_growth_rate', 'listed_growth_rate',
       'followers_friends_ratio', 'Name_entropy', 'Screen_name_entropy',
       'Name_freq', 'Screen_name_freq', 'Name_similarity', '_url',
       '_location'
]
for dataset in range(1,16):
  data = pd.read_csv(f'PATH/DS{dataset}.csv.gz',compression='gzip',lineterminator='\n');
  data = data[columns]
  data.to_csv(f'PATH/DS{dataset}.csv.gz', compression='gzip',index=False)